In [38]:
!pip install evaluate
!pip install peft

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


^C
ERROR: Operation cancelled by user


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Import libraries and load Dataset

In [39]:
from datasets import load_dataset, DatasetDict, Dataset
from transformers import (

    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np


In [40]:
model_checkpoint = "distilbert-base-uncased"

id2label = {0:"negative",1:"neutral",2:"positive"}
label2id = {"negative":0,"neutral":1,"positive":2}

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,num_labels=3,id2label=id2label,label2id=label2id)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
dataset = load_dataset("Sp1786/multiclass-sentiment-analysis-dataset")

In [42]:
#Dataset shape
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'sentiment'],
        num_rows: 31232
    })
    validation: Dataset({
        features: ['id', 'text', 'label', 'sentiment'],
        num_rows: 5205
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'sentiment'],
        num_rows: 5206
    })
})

In [43]:
dataset2 = DatasetDict({
    "validation": dataset["validation"],
    "test": dataset["test"]
})
dataset2

DatasetDict({
    validation: Dataset({
        features: ['id', 'text', 'label', 'sentiment'],
        num_rows: 5205
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'sentiment'],
        num_rows: 5206
    })
})

# Tokenizer

In [44]:
#Define the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,add_prefix_space=True)

In [45]:
#model.print_trainable_parameters()

# Model Performance before fine-tuning/ From raw LLM

In [46]:
#Untrained Model Performance
text_examples = ["That food was absolutely delicious.","I hate playing chess.","I love watching football.","The food was okay","Oh of course you don't know how it works."]

for text in text_examples:
    #tokenize text
    input = tokenizer.encode(text,return_tensors="pt")
    #compute logits
    logit = model(input).logits
    predictions = torch.argmax(logit)
    print(text+"-"+id2label[predictions.tolist()])


That food was absolutely delicious.-negative
I hate playing chess.-positive
I love watching football.-positive
The food was okay-neutral
Oh of course you don't know how it works.-negative


In [47]:
#Tokenize entire dataset

def tokenize_function(examples):
    texts = examples["text"]

    #tokenize and truncate
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        texts,
        return_tensors = "np",
        truncation = True,
        max_length = 512,
        padding=False
    )
    return tokenized_inputs
#Adding PAD tokens if none exist    
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({"pad_token":"[PAD]"})
    model.resize_token_embeddings(len(tokenizer))

#tokenize training(validation here, due to training set having way too many samples and testing datasets)
#tokenized_datasets = dataset2["validation"].map(tokenize_function,batched=True)

# DEBUGGING ERROR

In [48]:
#tokenized_datasets

In [49]:
import numpy as np
import pandas as pd
df = pd.DataFrame(dataset2["test"])
df["text"].isna().sum()

1

In [50]:
dataset2["test"] = dataset2["test"].filter(lambda example: pd.notna(example["text"]))


In [51]:
dataset2

DatasetDict({
    validation: Dataset({
        features: ['id', 'text', 'label', 'sentiment'],
        num_rows: 5205
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'sentiment'],
        num_rows: 5205
    })
})

In [52]:
#Tokenize entire dataset

def tokenize_function(examples):
    texts = examples["text"]

    #tokenize and truncate
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        texts,
        return_tensors = "np",
        truncation = True,
        max_length = 512,
        padding=False
    )
    return tokenized_inputs
#Adding PAD tokens if none exist    
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({"pad_token":"[PAD]"})
    model.resize_token_embeddings(len(tokenizer))

#tokenize training(validation here, due to training set having way too many samples and testing datasets)
tokenized_datasets = dataset2.map(tokenize_function,batched=True)

Map:   0%|          | 0/5205 [00:00<?, ? examples/s]

In [53]:
tokenized_datasets

DatasetDict({
    validation: Dataset({
        features: ['id', 'text', 'label', 'sentiment', 'input_ids', 'attention_mask'],
        num_rows: 5205
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'sentiment', 'input_ids', 'attention_mask'],
        num_rows: 5205
    })
})

# ERROR RESOLVED, one set of data had 1 null value

In [54]:
#Data collator for efficient and dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# MODEL

In [55]:
#Define evaluation metrics

accuracy = evaluate.load("accuracy")

#An evaluation function to pass into trainer
def compute_metrics(p):
    predictions,labels = p
    predictions = np.argmax(predictions,axis=1)

    return {"accuracy":accuracy.compute(predictions=predictions,references=labels)}
    

In [56]:
#PEFT using LORA for finetuning

peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,#intrinsic rank or rank of decomposition
                        lora_alpha=32,
                        lora_dropout = 0.01,
                        target_modules=["q_lin"])


In [57]:
model = get_peft_model(model,peft_config)
model.print_trainable_parameters()

trainable params: 629,763 || all params: 67,585,542 || trainable%: 0.9318


In [58]:
# hyperparameters
lr = 1e-3  # size of optimization step
batch_size = 4  # number of examples processed per optimization step
num_epochs = 10  # number of times model runs through training data

# define training arguments
training_args = TrainingArguments(
    output_dir = "/kaggle/working/" + model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [59]:
trainer = Trainer(
    model=model,  
    args=training_args, 
    train_dataset=tokenized_datasets["validation"],#using validation for training as actual training set is pretty heavy, we dont need those many samples  
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer, 
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


/tmp/ipykernel_23/1145778912.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [60]:
import os
os.environ["WANDB_DISABLED"] = "true"#I don't have a Weights&Biases api key :/

In [61]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,0.803900,0.727763,{'accuracy': 0.6806916426512968}
2,0.657800,0.700642,{'accuracy': 0.7014409221902017}
3,0.638500,0.783041,{'accuracy': 0.7064361191162344}
4,0.535400,0.823210,{'accuracy': 0.7125840537944285}
5,0.501100,0.851971,{'accuracy': 0.7056676272814602}
6,0.447400,0.982143,{'accuracy': 0.7068203650336216}
7,0.377200,1.006208,{'accuracy': 0.7093179634966379}
8,0.333800,1.080426,{'accuracy': 0.7081652257444765}
9,0.302900,1.128694,{'accuracy': 0.7116234390009606}
10,0.263200,1.165566,{'accuracy': 0.7121998078770413}


Trainer is attempting to log a value of "{'accuracy': 0.6806916426512968}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Trainer is attempting to log a value of "{'accuracy': 0.7014409221902017}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is inco

TrainOutput(global_step=6510, training_loss=0.475362392028539, metrics={'train_runtime': 805.101, 'train_samples_per_second': 64.65, 'train_steps_per_second': 8.086, 'total_flos': 888680796217200.0, 'train_loss': 0.475362392028539, 'epoch': 10.0})

In [72]:

"""
TrainOutput(global_step=6510, training_loss=0.475362392028539, metrics={'train_runtime': 805.101, 'train_samples_per_second': 64.65, 'train_steps_per_second': 8.086, 'total_flos': 888680796217200.0, 'train_loss': 0.475362392028539, 'epoch': 10.0})
"""

"\nTrainOutput(global_step=6510, training_loss=0.475362392028539, metrics={'train_runtime': 805.101, 'train_samples_per_second': 64.65, 'train_steps_per_second': 8.086, 'total_flos': 888680796217200.0, 'train_loss': 0.475362392028539, 'epoch': 10.0})\n"

# Testing out our Model

In [65]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

text_examples = [
    "That food was absolutely delicious.",
    "I hate playing chess.",
    "I love watching football.",
    "The food was okay",
    "Oh of course you don't know how it works."
]

for text in text_examples:
    # Tokenize text
    input = tokenizer.encode(text, return_tensors="pt").to(device)
    
    # Compute logits
    logit = model(input).logits
    
    # Compute predictions
    predictions = torch.argmax(logit, dim=-1)
    
    # Print results
    print(text + " - " + id2label[predictions.item()])


That food was absolutely delicious. - positive
I hate playing chess. - negative
I love watching football. - positive
The food was okay - neutral
Oh of course you don't know how it works. - negative
